In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

med_gemma_impact_challenge_path = kagglehub.competition_download('med-gemma-impact-challenge')

print('Data source import complete.')


# Agentic Health Triage for Rural Areas

This notebook demonstrates a minimal agentic workflow for basic health
triage designed for low-connectivity environments.


## Objective
Provide basic health orientation and urgency alerts using an offline-capable
agentic workflow built with open models.


# Core imports
import json
from typing import Dict


In [ ]:
from dataclasses import dataclass
from typing import List, Optional

@dataclass
class PatientInput:
    age: int
    symptoms: List[str]
    duration_days: int
    chronic_conditions: Optional[List[str]] = None
    pregnant: Optional[bool] = False
    temperature_c: Optional[float] = None


In [ ]:
sample_patient = PatientInput(
    age=67,
    symptoms=["febre", "tosse", "falta de ar"],
    duration_days=3,
    chronic_conditions=["hipertensao"],
    temperature_c=38.5
)

sample_patient

In [ ]:
#bloco 2 da fase 3 - criação do sistema de score


def calculate_risk_score(patient: PatientInput) -> int:
    score = 0

    # Idade
    if patient.age >= 65:
        score += 2

    # Febre
    if patient.temperature_c:
        if patient.temperature_c >= 39:
            score += 3
        elif patient.temperature_c >= 38:
            score += 2

    # Sintomas críticos
    if "falta de ar" in patient.symptoms:
        score += 3

    # Doenças crônicas
    if patient.chronic_conditions:
        score += 2

    # Duração prolongada
    if patient.duration_days > 7:
        score += 1

    return score



In [ ]:
def classify_risk(score: int) -> str:

    if score >= 7:
        return "CRITICAL"

    if score >= 5:
        return "HIGH"

    if score >= 3:
        return "MEDIUM"

    return "LOW"


In [ ]:
score = calculate_risk_score(sample_patient)
risk_level = classify_risk(score)

print("Risk Score:", score)
print("Risk Level:", risk_level)


In [ ]:
#bloco 03

def generate_risk_explanation(patient: PatientInput, score: int) -> str:

    reasons = []

    if patient.age >= 65:
        reasons.append("Idade avançada (≥65 anos)")

    if patient.temperature_c:
        if patient.temperature_c >= 39:
            reasons.append("Febre alta (≥39°C)")
        elif patient.temperature_c >= 38:
            reasons.append("Febre moderada (≥38°C)")

    if "falta de ar" in patient.symptoms:
        reasons.append("Presença de falta de ar")

    if patient.chronic_conditions:
        reasons.append("Histórico de condições crônicas")

    if patient.duration_days > 7:
        reasons.append("Sintomas persistentes por mais de 7 dias")

    explanation = "Classificação baseada nos seguintes fatores: " + ", ".join(reasons)

    return explanation


In [ ]:
explanation = generate_risk_explanation(sample_patient, score)
print(explanation)


In [ ]:
# bloco 04

def generate_recommendation(risk_level: str) -> str:

    if risk_level == "CRITICAL":
        return "Encaminhamento imediato para emergência. Procurar atendimento hospitalar urgentemente."

    if risk_level == "HIGH":
        return "Recomendado atendimento médico nas próximas horas. Monitorar sinais vitais e sintomas."

    if risk_level == "MEDIUM":
        return "Monitoramento domiciliar com orientação médica remota. Retornar se houver piora."

    return "Baixo risco. Manter observação e cuidados gerais."


In [ ]:
recommendation = generate_recommendation(risk_level)

print("Risk Score:", score)
print("Risk Level:", risk_level)
print("Explanation:", explanation)
print("Recommendation:", recommendation)


# FASE 4- Arquitetura de agente de triagem inteligente

Esta fase transfora a lógica de triagem em um agente orquestrado com saída estruturada, simulação de múltiplos pacientes e análises simples.


In [ ]:
from datetime import datetime

In [ ]:
import uuid

class HealthTriageAgent:

    def __init__(self):
        self.history = []

    def validate_input(self, patient):
        if patient.age < 0:
            raise ValueError("Age cannot be negative")
        return True

    def calculate_score(self, patient):
        score = 0

        if patient.age >= 65:
            score += 2

        if patient.temperature_c is not None and patient.temperature_c >= 38:
            score += 2

        if "falta de ar" in patient.symptoms:
            score += 3

        if patient.chronic_conditions:
            score += 2

        if patient.duration_days > 7:
            score += 1

        return score

    def classify(self, score):
        if score >= 7:
            return "CRITICAL"
        elif score >= 5:
            return "HIGH"
        elif score >= 3:
            return "MEDIUM"
        else:
            return "LOW"

    def explain(self, patient):
        reasons = []

        if patient.age >= 65:
            reasons.append("Idade avançada")

        if patient.temperature_c is not None and patient.temperature_c >= 38:
            reasons.append("Febre")

        if "falta de ar" in patient.symptoms:
            reasons.append("Falta de ar")

        if patient.chronic_conditions:
            reasons.append("Condições crônicas")

        if patient.duration_days > 7:
            reasons.append("Sintomas prolongados")

        if not reasons:
            return "Nenhum fator de risco relevante identificado."

        return "Fatores identificados: " + ", ".join(reasons)

    def recommend(self, classification):
        if classification == "CRITICAL":
            return "Encaminhamento imediato para emergência."
        elif classification == "HIGH":
            return "Atendimento médico nas próximas horas."
        elif classification == "MEDIUM":
            return "Monitoramento domiciliar."
        else:
            return "Baixo risco."
    def run(self, patient):
        self.validate_input(patient)

        score = self.calculate_score(patient)
        classification = self.classify(score)
        explanation = self.explain(patient)
        recommendation = self.recommend(classification)

        result = {
            "timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
            "patient_id": str(uuid.uuid4())[:8],
            "risk_score": score,
            "risk_level": classification,
            "explanation": explanation,
            "recommended_action": recommendation
         }

        self.history.append(result)

        return result


    def get_history(self):
        return self.history


#paramos na fase 4 na ajeitacao da identacao , dps continuarmos






In [ ]:
agent = HealthTriageAgent()

result = agent.run(sample_patient)

agent.get_history()

result


In [ ]:
class Patient:
    def __init__(self, age, temperature_c, symptoms, chronic_conditions, duration_days):
        self.age = age
        self.temperature_c = temperature_c
        self.symptoms = symptoms
        self.chronic_conditions = chronic_conditions
        self.duration_days = duration_days

sample_patient = Patient(
    age=70,
    temperature_c=38.5,
    symptoms=["falta de ar", "tosse"],
    chronic_conditions=["hipertensão"],
    duration_days=5
)




In [ ]:
patients = [
    Patient(70, 38.5, ["falta de ar", "tosse"], ["hipertensão"], 5),
    Patient(25, 37.0, ["dor de cabeça"], [], 2),
    Patient(80, 39.0, ["falta de ar"], ["diabetes"], 10),
    Patient(45, 38.2, ["tosse"], [], 4),
    Patient(60, 36.8, ["cansaço"], ["asma"], 8)
]

In [ ]:
results = [agent.run(p) for p in patients]

results


In [ ]:
from collections import Counter

In [ ]:
risk_distribution = Counter([r["risk_level"] for r in results])

risk_distribution


In [ ]:
import pandas as pd
import random

# -----------------------------
# Listas base
# -----------------------------

sintomas = [
    "febre",
    "dor de cabeça",
    "dor no peito",
    "tosse",
    "vomito",
    "falta de ar",
    "diarreia",
    "dor abdominal"
]

intensidades = ["baixa", "media", "alta"]

condicoes = [
    "nenhuma",
    "diabetes",
    "hipertensao",
    "asma",
    "cardiopatia"
]

# -----------------------------
# Definir grupo de risco
# -----------------------------
def definir_grupo_risco(idade, condicao):

    if idade >= 60:
        return "sim"

    if condicao != "nenhuma":
        return "sim"

    return "nao"

# -----------------------------
# Definir nível de risco
# -----------------------------
def definir_nivel_risco(idade, sintoma, intensidade, duracao, grupo_risco):

    # Casos críticos imediatos
    if sintoma == "dor no peito" and intensidade == "alta":
        return "alto"

    if sintoma == "falta de ar":
        return "alto"

    if idade >= 60 and sintoma == "febre" and intensidade == "alta":
        return "alto"

    if grupo_risco == "sim" and intensidade == "alta":
        return "alto"

    # Casos intermediários
    if duracao >= 5:
        return "medio"

    if grupo_risco == "sim" and intensidade == "media":
        return "medio"

    if intensidade == "alta":
        return "medio"

    # Casos leves
    return "baixo"

# -----------------------------
# Geração dos dados
# -----------------------------

dados = []

for _ in range(80):  # aumentei para mais robustez

    idade = random.randint(1, 85)
    sintoma = random.choice(sintomas)
    intensidade = random.choice(intensidades)
    duracao = random.randint(1, 7)

    condicao = random.choices(
        condicoes,
        weights=[0.6, 0.1, 0.1, 0.1, 0.1]  # maioria sem condição
    )[0]

    grupo_risco = definir_grupo_risco(idade, condicao)

    nivel_risco = definir_nivel_risco(
        idade,
        sintoma,
        intensidade,
        duracao,
        grupo_risco
    )

    dados.append({
        "IDADE": idade,
        "SINTOMA_PRINCIPAL": sintoma,
        "INTENSIDADE": intensidade,
        "DURACAO_DIAS": duracao,
        "CONDICAO_PREEXISTENTE": condicao,
        "GRUPO_RISCO": grupo_risco,
        "NIVEL_RISCO": nivel_risco
    })

df = pd.DataFrame(dados)

df.head()


In [ ]:
# Core imports
import json
from typing import Dict


In [ ]:
def interpret_symptoms(user_input: str) -> Dict:
    return {
        "interpreted_symptoms": user_input.lower(),
        "confidence": "basic"
    }


In [ ]:
def classify_risk(symptoms: Dict) -> str:
    text = symptoms["interpreted_symptoms"]
    if "dor no peito" in text or "falta de ar" in text:
        return "high"
    elif "febre" in text:
        return "medium"
    return "low"


In [ ]:
def provide_guidance(risk_level: str) -> str:
    if risk_level == "high":
        return "Procure atendimento médico imediato."
    elif risk_level == "medium":
        return "Observe os sintomas e procure um profissional se persistirem."
    return "Repouso e hidratação podem ajudar."


In [ ]:
def agentic_triage_pipeline(user_input: str) -> Dict:
    symptoms = interpret_symptoms(user_input)
    risk = classify_risk(symptoms)
    guidance = provide_guidance(risk)

    return {
        "risk_level": risk,
        "guidance": guidance
    }


In [ ]:
user_input = "Estou com febre e dor no corpo"
result = agentic_triage_pipeline(user_input)
result
